# How to generate error bars for 2Q-GST


In [1]:
import pygsti
import pickle
import time

#If we were using MPI
# from mpi4py import MPI
# comm = MPI.COMM_WORLD
comm = None

#Load the 2-qubit results (if you don't have this file, run the 2Q-GST example)
with open("example_files/easy_2q_results.pkl","rb") as f:
    results = pickle.load(f)

#Set a memory limit
print("Memory limit was = ", results.estimates['default'].parameters.get('memLimit',"none given"))
results.estimates['default'].parameters['memLimit'] = 2.5*(1024.0)**3 # 2.5GB
print("Memory limit is now = ", results.estimates['default'].parameters['memLimit'])

Memory limit was =  3221225472
Memory limit is now =  2684354560.0


In [2]:
# error bars in reports require the presence of a fully-initialized
# "confidence region factory" within the relevant Estimate object.
# In most cases "fully-initialized" means that a Hessian has been 
# computed and projected onto the non-gauge space.
start = time.time()

# initialize a factory for the 'go0' gauge optimization within the 'default' estimate
crfact = results.estimates['default'].add_confidence_region_factory('go0', 'final')
crfact.compute_hessian(comm=comm) #optionally use multiple processors
crfact.project_hessian('intrinsic error')

end = time.time()
print("Total time=%f hours" % ((end - start) / 3600.0))

Evaltree generation (deriv) w/mem limit = 2.48GB
 mem(1 subtrees, 1,1 param-grps, 1 proc-grps) in 0s = 6773.17GB (6773.17GB fc)
Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
 groups of ~1 procs each, to distribute over (1616,1616) params (taken as 1616,4 param groups of ~1,404 params).
 Memory estimate = 2.08GB (cache=1317, wrtLen1=1, wrtLen2=404, subsPerProc=1).
rank 0: 38.7419s: block 0/4043, sub-tree 0/1, sub-tree-len = 1317
rank 0: 120.189s: block 1/4043, sub-tree 0/1, sub-tree-len = 1317
rank 0: 178.593s: block 2/4043, sub-tree 0/1, sub-tree-len = 1317
rank 0: 234.334s: block 3/4043, sub-tree 0/1, sub-tree-len = 1317
rank 0: 290.545s: block 4/4043, sub-tree 0/1, sub-tree-len = 1317
rank 0: 338.526s: block 5/4043, sub-tree 0/1, sub-tree-len = 1317
rank 0: 385.31s: block 6/4043, sub-tree 0/1, sub-tree-len = 1317
rank 0: 427.241s: block 7/4043, sub-tree 0/1, sub-tree-len = 1317
rank 0: 463.751s: block 8/4043, sub-tree 0/1, sub-tree-len = 13

KeyboardInterrupt: 

Note above cell was executed for demonstration purposes, and was **keyboard-interrupted intentionally** since it would have taken forever on a single processor.

In [ ]:
#write results back to file
with open("example_files/easy_2q_results_withCI.pkl","wb") as f:
    pickle.dump(results, f)